In [1]:
from fds.quant.universe import IdentifierUniverse
from fds.quant.fql import FQL, FQLExpression
from fds.quant.dates import TimeSeries
from fds.quant.universe import ScreeningExpressionUniverse
from fds.quant.screening import Screen, ScreeningExpression
from fds.quant._data import _executor as executor

import pandas as pd
import numpy as np
import datetime as datetime
from functools import reduce
import ipywidgets as widgets
from IPython.display import display

In [2]:
index_tuple = [ # Name, Ticker, Index/ETF
    ('NASDAQ 100', 'QQQ-US', 'ETF'),
    ('S&P 500', 'SP50', 'Index'),
    ('Russel 1000', 'R.1000','Index') 
] 

start_date = '20020101'

date_julian = {
    'Date' : FQLExpression(expression=f'JULIAN(FG_PRICE({start_date},0,M).dates)', name='1', is_array=True)
}
EPS_up = {
    'EPS Est Rev UP Q1'  : FQLExpression(expression=f'FE_ESTIMATE(EPS,UP,QTR_ROLL,+1,{start_date},0,M)', name='1', is_array=True),
    'EPS Est Rev UP Q2'  : FQLExpression(expression=f'FE_ESTIMATE(EPS,UP,QTR_ROLL,+2,{start_date},0,M)', name='1', is_array=True),
    'EPS Est Rev UP Q3'  : FQLExpression(expression=f'FE_ESTIMATE(EPS,UP,QTR_ROLL,+3,{start_date},0,M)', name='1', is_array=True),
    'EPS Est Rev UP Q4'  : FQLExpression(expression=f'FE_ESTIMATE(EPS,UP,QTR_ROLL,+4,{start_date},0,M)', name='1', is_array=True)
}
EPS_down = {
    'EPS Est Rev DOWN Q1'  : FQLExpression(expression=f'FE_ESTIMATE(EPS,DOWN,QTR_ROLL,+1,{start_date},0,M)', name='1', is_array=True),
    'EPS Est Rev DOWN Q2'  : FQLExpression(expression=f'FE_ESTIMATE(EPS,DOWN,QTR_ROLL,+2,{start_date},0,M)', name='1', is_array=True),
    'EPS Est Rev DOWN Q3'  : FQLExpression(expression=f'FE_ESTIMATE(EPS,DOWN,QTR_ROLL,+3,{start_date},0,M)', name='1', is_array=True),
    'EPS Est Rev DOWN Q4'  : FQLExpression(expression=f'FE_ESTIMATE(EPS,DOWN,QTR_ROLL,+4,{start_date},0,M)', name='1', is_array=True)
}
EPS_nest = {
    'EPS Est Rev NEST Q1'  : FQLExpression(expression=f'FE_ESTIMATE(EPS,NEST,QTR_ROLL,+1,{start_date},0,M)', name='1', is_array=True),
    'EPS Est Rev NEST Q2'  : FQLExpression(expression=f'FE_ESTIMATE(EPS,NEST,QTR_ROLL,+2,{start_date},0,M)', name='1', is_array=True),
    'EPS Est Rev NEST Q3'  : FQLExpression(expression=f'FE_ESTIMATE(EPS,NEST,QTR_ROLL,+3,{start_date},0,M)', name='1', is_array=True),
    'EPS Est Rev NEST Q4'  : FQLExpression(expression=f'FE_ESTIMATE(EPS,NEST,QTR_ROLL,+4,{start_date},0,M)', name='1', is_array=True),
}

In [3]:
%%time
ts = TimeSeries(start = start_date, freq = 'M')
ticker = [x[1] for x in index_tuple]
ticker_type = [x[2] for x in index_tuple]

# PIT Constituents
constituent = {}
TrueFalse = pd.DataFrame()
command = "executor.calculate(["
for c in range(len(ticker)):
    if ticker_type[c] == 'Index':
        expression = ["FG_CONSTITUENTS(" + ticker[c] + ",0,CLOSE)=1"] 
    elif ticker_type[c] == 'ETF':
        expression = ["PMWU(LION:" + ticker[c] + ",0)=1"] 
    else: 
        raise ValueError(f"{ticker_type[c]} is neither ETF nor Index. Please fix this.")
    constituent[c] = ScreeningExpressionUniverse(expression = expression, time_series = ts)
    command = command + "constituent["+str(c)+"],"    
command = command[:-1] + "])"
eval(command) # executor can run multiple runs (max 4) in parallel 

for c in range(len(ticker)):
    constituent_pd = constituent[c].constituents.drop(columns=['company_name', 'ticker'])
    constituent_pd['index'] = ticker[c]
    TrueFalse = TrueFalse.append(constituent_pd)
    
TrueFalse = (TrueFalse.fillna(value=False)) * 1
TrueFalse = TrueFalse.reset_index().set_index(['symbol','index']).reset_index()
TrueFalse['1_0_list'] = TrueFalse.to_numpy()[:,2:].tolist()
Reduced = TrueFalse.filter(['index','symbol','1_0_list'])

['PMWU(LION:QQQ-US,0)=1']: 0it [00:00, ?it/s]

['FG_CONSTITUENTS(R.1000,0,CLOSE)=1']: 0it [00:00, ?it/s]

['FG_CONSTITUENTS(SP50,0,CLOSE)=1']: 0it [00:00, ?it/s]

Generating DataFrame: 0it [00:00, ?it/s]

Generating DataFrame: 0it [00:00, ?it/s]

Generating DataFrame: 0it [00:00, ?it/s]

CPU times: user 4.43 s, sys: 331 ms, total: 4.76 s
Wall time: 1min 30s


In [4]:
%%time
# Constituents data 
constituents_unique = IdentifierUniverse(list(Reduced['symbol'].unique()))

date_fql = FQL(universe = constituents_unique, formulas = date_julian.values(), columns = date_julian.keys())
EPS_up_fql = FQL(universe = constituents_unique, formulas = EPS_up.values(), columns = EPS_up.keys())
EPS_down_fql = FQL(universe = constituents_unique, formulas = EPS_down.values(), columns = EPS_down.keys())
EPS_nest_fql = FQL(universe = constituents_unique, formulas = EPS_nest.values(), columns = EPS_nest.keys())

executor.calculate([date_fql, EPS_up_fql, EPS_down_fql, EPS_nest_fql])

fql_arrays = [date_fql.data.drop(['ison_univ','company_name','ticker'],axis=1), 
              EPS_up_fql.data.drop(['ison_univ','company_name','ticker'],axis=1), 
              EPS_down_fql.data.drop(['ison_univ','company_name','ticker'],axis=1), 
              EPS_nest_fql.data.drop(['ison_univ','company_name','ticker'],axis=1)]

merged_fql_array = reduce(lambda  left,right: pd.merge(left, right, left_index=True, right_index=True, how='outer'), fql_arrays)
merged_fql_array = merged_fql_array.droplevel(level=0)

FQL: 0it [00:00, ?it/s]

FQL: 0it [00:00, ?it/s]

FQL: 0it [00:00, ?it/s]

FQL: 0it [00:00, ?it/s]

Generating DataFrame: 0it [00:00, ?it/s]

Generating DataFrame: 0it [00:00, ?it/s]

Generating DataFrame: 0it [00:00, ?it/s]

Generating DataFrame: 0it [00:00, ?it/s]

CPU times: user 579 ms, sys: 138 ms, total: 717 ms
Wall time: 37.4 s


In [6]:
TF_FQL

1_0_list       Date  EPS Est Rev UP Q1  EPS Est Rev UP Q2  \
index  symbol                                                                
R.1000 00081T10         1 2005-08-31                0.0                0.0   
       00081T10         1 2005-09-30                0.0                0.0   
       00081T10         1 2005-10-31                0.0                0.0   
       00081T10         1 2005-11-30                0.0                0.0   
       00081T10         1 2005-12-30                0.0                0.0   
...                   ...        ...                ...                ...   
       Y8564W10         1 2016-01-29                0.0                0.0   
       Y8564W10         1 2016-02-29                0.0                1.0   
       Y8564W10         1 2016-03-31                0.0                1.0   
       Y8564W10         1 2016-04-29                0.0                1.0   
       Y8564W10         1 2016-05-31                2.0                2.0   

                 EPS Est Rev UP Q3  EPS Est Rev UP Q4  EPS Est Rev DOWN Q1  \
index  symbol                                                                
R.1000 00081T10                0.0                0.0                  0.0   
       00081T10                0.0                0.0                  0.0   
       00081T10                0.0                0.0                  0.0   
       00081T10                0.0                0.0                  0.0   
       00081T10                0.0                0.0                  0.0   
...                            ...                ...                  ...   
       Y8564W10                2.0                2.0                  3.0   
       Y8564W10                1.0                4.0                  4.0   
       Y8564W10                1.0                4.0                  4.0   
       Y8564W10                2.0                4.0                  4.0   
       Y8564W10                2.0                1.0                  1.0   

                 EPS Est Rev DOWN Q2  EPS Est Rev DOWN Q3  \
index  symbol                                               
R.1000 00081T10                  0.0                  0.0   
       00081T10                  0.0                  0.0   
       00081T10                  0.0                  0.0   
       00081T10                  0.0                  0.0   
       00081T10                  0.0                  0.0   
...                              ...                  ...   
       Y8564W10                  1.0                  0.0   
       Y8564W10                  3.0                  2.0   
       Y8564W10                  3.0                  2.0   
       Y8564W10                  3.0                  2.0   
       Y8564W10                  1.0                  1.0   

                 EPS Est Rev DOWN Q4  EPS Est Rev NEST Q1  \
index  symbol                                               
R.1000 00081T10                  0.0                  0.0   
       00081T10                  0.0                  0.0   
       00081T10                  0.0                  0.0   
       00081T10                  0.0                  0.0   
       00081T10                  0.0                  0.0   
...                              ...                  ...   
       Y8564W10                  0.0                  7.0   
       Y8564W10                  0.0                  7.0   
       Y8564W10                  0.0                  4.0   
       Y8564W10                  0.0                  5.0   
       Y8564W10                  1.0                  4.0   

                 EPS Est Rev NEST Q2  EPS Est Rev NEST Q3  EPS Est Rev NEST Q4  
index  symbol                                                                   
R.1000 00081T10                  0.0                  0.0                  0.0  
       00081T10                  0.0                  0.0                  0.0  
       00081T10                  0.0                  0.0                  

In [5]:
%%time
# Merge dfs using the index symbol as index. Use the df where tickers are repeating and joining the df with unique tickers
TF_FQL = Reduced.set_index('symbol').join(merged_fql_array, how='left').reset_index().set_index(['index','symbol'])

# Sanity check on array length before explode
def check_list_lengths(df, col_names):
    for col in col_names:
        length = df[col].apply(len).unique()
        if len(length) > 1:
            raise ValueError(f"Column {col} has lists of different lengths. Please fix this or your code will run infinitely while exploding array elements")

col_names = TF_FQL.columns.tolist()
check_list_lengths(TF_FQL, col_names)

print("Ready to explode array")

# Explode array values
TF_FQL['combined'] = TF_FQL.apply(lambda x: list(zip(*[x[col] for col in col_names])), axis=1)
TF_FQL = TF_FQL.explode('combined')
TF_FQL[col_names] = pd.DataFrame(TF_FQL['combined'].tolist(), index=TF_FQL.index)
TF_FQL = TF_FQL.drop(columns='combined')

# Convert Factset Julian date
TF_FQL.Date = pd.to_datetime(TF_FQL.Date.astype('int64'), origin='1899-12-30', unit='D')

TF_FQL = TF_FQL[TF_FQL['1_0_list']==1]

print("Array explode success")

Ready to explode array
Array explode success
CPU times: user 6.05 s, sys: 1.36 s, total: 7.41 s
Wall time: 7.38 s


In [6]:
%%time
TF_FQL_Agg = {}
col_names = [col for col in col_names if col not in ['Date', '1_0_list']]
for col in col_names:
    TF_FQL_Agg[col] = TF_FQL.groupby(['Date', 'index'])[col].sum()

if TF_FQL_Agg:
    # Initialize the merged_df with the first dataframe
    EPS_revision = next(iter(TF_FQL_Agg.values()))

    # Merge all other dataframes
    for df_name, df in list(TF_FQL_Agg.items())[1:]:
        EPS_revision = pd.merge(EPS_revision, df, on=['Date', 'index'], how='outer')

EPS_revision

# Revision breadth
EPS_revision['EPS Est Rev UP FY'] = EPS_revision['EPS Est Rev UP Q1'] + EPS_revision['EPS Est Rev UP Q2'] + EPS_revision['EPS Est Rev UP Q3'] + EPS_revision['EPS Est Rev UP Q4']
EPS_revision['EPS Est Rev DOWN FY'] = EPS_revision['EPS Est Rev DOWN Q1'] + EPS_revision['EPS Est Rev DOWN Q2'] + EPS_revision['EPS Est Rev DOWN Q3'] + EPS_revision['EPS Est Rev DOWN Q4']
EPS_revision['EPS Est Rev NEST FY'] = EPS_revision['EPS Est Rev NEST Q1'] + EPS_revision['EPS Est Rev NEST Q2'] + EPS_revision['EPS Est Rev NEST Q3'] + EPS_revision['EPS Est Rev NEST Q4']
EPS_revision['EPS Est Rev Breadth'] = (EPS_revision['EPS Est Rev UP FY'] - EPS_revision['EPS Est Rev DOWN FY']) / EPS_revision['EPS Est Rev NEST FY']

EPS_revision

CPU times: user 459 ms, sys: 7.68 ms, total: 467 ms
Wall time: 465 ms


EPS Est Rev UP Q1  EPS Est Rev UP Q2  EPS Est Rev UP Q3  \
Date       index                                                             
2002-01-31 QQQ-US               11.0                6.0                9.0   
           R.1000               69.0               67.0               75.0   
           SP50                 48.0               47.0               60.0   
2002-02-28 QQQ-US               37.0               31.0               31.0   
           R.1000              296.0              275.0              292.0   
...                              ...                ...                ...   
2024-04-30 R.1000             4246.0             4131.0             4338.0   
           SP50               2508.0             2391.0             2520.0   
2024-05-31 QQQ-US              897.0              709.0              724.0   
           R.1000             4982.0             4529.0             4752.0   
           SP50               2803.0             2674.0             2797.0   

                   EPS Est Rev UP Q4  EPS Est Rev DOWN Q1  \
Date       index                                            
2002-01-31 QQQ-US                7.0                 15.0   
           R.1000               76.0                130.0   
           SP50                 60.0                 87.0   
2002-02-28 QQQ-US               28.0                 50.0   
           R.1000              333.0                340.0   
...                              ...                  ...   
2024-04-30 R.1000             3890.0               5452.0   
           SP50               2164.0               3266.0   
2024-05-31 QQQ-US              609.0                886.0   
           R.1000             3692.0               6575.0   
           SP50               2167.0               3978.0   

                   EPS Est Rev DOWN Q2  EPS Est Rev DOWN Q3  \
Date       index                                              
2002-01-31 QQQ-US                 16.0                 13.0   
           R.1000                100.0                 82.0   
           SP50                   69.0                 50.0   
2002-02-28 QQQ-US                 56.0                 46.0   
           R.1000                301.0                268.0   
...                                ...                  ...   
2024-04-30 R.1000               5018.0               4664.0   
           SP50                 3010.0               2762.0   
2024-05-31 QQQ-US                919.0                855.0   
           R.1000               6290.0               5995.0   
           SP50                 3608.0               3390.0   

                   EPS Est Rev DOWN Q4  EPS Est Rev NEST Q1  \
Date       index                                              
2002-01-31 QQQ-US                  8.0                252.0   
           R.1000                 68.0               1876.0   
           SP50                   46.0               1224.0   
2002-02-28 QQQ-US                 29.0                373.0   
           R.1000                229.0               2649.0   
...                                ...                  ...   
2024-04-30 R.1000               2667.0              14409.0   
           SP50                 1421.0               8516.0   
2024-05-31 QQQ-US                402.0               2306.0   
           R.1000               2523.0              14338.0   
           SP50                 1380.0               8438.0   

                   EPS Est Rev NEST Q2  EPS Est Rev NEST Q3  \
Date       index                                              
2002-01-31 QQQ-US                228.0                204.0   
           R.1000               1554.0               1447.0   
           SP50                 1039.0                964.0   
2002-02-28 QQQ-US                358.0                327.0   
           R.1000               2481.0               2359.0   
...                                ...                  ...   
2024-04-30 R.1000              14135.0              13876.0   
   

In [21]:
EPS_revision.to_csv("EPS_revision_breadth.csv")